<a href="https://colab.research.google.com/github/GuilhermePelegrina/Mackenzie/blob/main/Aulas/2025_1s/An%C3%A1lise%20de%20Dados/Aula_07_Series_temporais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="Universidade Presbiteriana Mackenzie">
</head>

<img src="http://meusite.mackenzie.br/rogerio/mackenzie_logo/UPM.2_horizontal_vermelho.jpg" width=300, align="left">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->

# **Trabalhando com Séries Temporais**

Nesta aula, abordaremos conceitos introdutórios para manipulação e análise de séries temporais. Esses conceitos serão úteis para a disciplina de TIC no próximo semestre, onde abordaremos com mairo detalhe a construção de modelos de previsão de séries temporais.

Uma série temporal é uma sequência de observações ou medidas feitas em um mesmo objeto ou fenômeno em intervalos de tempo igualmente espaçados. Essas observações são coletadas e registradas em função do tempo. Em outras palavras, uma série temporal é uma coleção de dados organizados ao longo do tempo, o que permite analisar tendências, padrões, sazonalidades e outros comportamentos que possam ocorrer em diferentes escalas de tempo.

<img src='https://raw.githubusercontent.com/guilhermepelegrina/Mackenzie/main/Aulas/Figuras/fig_series_temporais_1.png' width="600">

As séries temporais são amplamente usadas em várias disciplinas, como econometria, meteorologia, finanças, engenharia e muitas outras, para analisar e prever o comportamento de fenômenos que evoluem com o tempo. Elas desempenham um papel fundamental na modelagem estatística e na tomada de decisões com base no histórico de dados.

Exemplos de aplicação na Engenharia de Produção são os seguintes:

- Previsão de demanda/vendas, com base em dados históricos,

- Otimização de estoque (necessidade de material futuro, por exemplo),

- Munutenção preditiva, de acordo com históricos de falhas de equipamentos.

Para os conceitos abordados nessa aula, utilizaremos como base de dados o histórico de produção mensal de cerveja na Austrália entre os anos de 1956 e 1995. O código abaixo faz a leitura dessa base de dados. Repare nos comandos inseridos. Eles possuem as seguintes funções:

- `parse_dates=['nome_coluna']`: Faz a conversão da informação contina na coluna `nome_coluna` para o tipo *datatime*. Como nesse conjunto de dados as informações são mensais, não há o dia específico. Por isso, o Python preencheu com *01*. No entanto, se os dados fossem diários, a conversão também explicitaria os dias. Em nosso caso, podemos indicar para o Python que queremos apenas as informações sobre os anos e dias.

- `index_col = ['nome_coluna']`: Converte a coluna `nome_coluna` para os índices da série temporal. Note que, com isso, não existe mais essa coluna no DataFrame, uma vez que ela apenas faz o papel de índice agora.

Além desses comandos, veja na sequência a criação de duas novas colunas contendo as informações sobre os anos e sobre os meses.

In [ ]:
# Lendo os dados (sem os dias)
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/ejgao/Time-Series-Datasets/master/monthly-beer-production-in-austr.csv", parse_dates=['Month'], index_col='Month')
df['Mês'] = df.index.month
df['Ano'] = df.index.year
df

Para uma primeira análise da série temporal, vamos visualizar seu comportamento ao longo do tempo.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(14,5))

plt.plot(df["Monthly beer production"])
plt.title("Podução mensal de cerveja")
plt.xlabel('Data')
plt.ylabel('Produção')
plt.show()

Nota-se que a produção cresceu ao longo do tempo (até 1980, principalmente), e depois se manteve com uma certa estabilidade. Além disso, podemos perceber uma sazonalidade nos dados. Vamos plotar esses dados para os anos de 1956 e 1958, para avaliar a caracaterística sazonal.

In [ ]:
plt.figure(figsize=(14,5))

plt.plot(df[(df['Ano']>=1956) & (df['Ano']<=1958)]["Monthly beer production"])
plt.title("Podução média mensal de cerveja")
plt.xlabel('Data')
plt.ylabel('Produção')
plt.show()

A partir desse gráfico, já temos a informação que os máximos ocorrem no final/começo do ano e que os mínimos ocorrem noa metade do ano. O que faz sentido, uma vez que os meses mais quentes do ano no Austrália são os do final/começo do ano.

Podemos verificar melhor essa constatação a partir da média ou distribuição da produção mensal de cerveja. Veja como fica essa análise nos códigos/gráficos abaixo.

In [ ]:
import seaborn as sns

# Produção média

media_mensal = pd.DataFrame(df.groupby('Mês')['Monthly beer production'].mean())

plt.figure(figsize=(14,5))

plt.plot(media_mensal)
plt.title("Podução mensal média de cerveja")
plt.xlabel('Mês')
plt.ylabel('Produção')
plt.show()

# Distribuição da produção
plt.figure(figsize=(14,5))

sns.boxplot(x="Mês", y="Monthly beer production", data=df)
plt.show()

Vamos, agora, analisar a produção de cerveja média por ano (para verificar se há alguma tendência). Veja como fica essa análise nos códigos/gráficos abaixo.

In [ ]:
# Produção média

media_anual = pd.DataFrame(df.groupby('Ano')['Monthly beer production'].mean())

plt.figure(figsize=(14,5))

plt.plot(media_anual)
plt.title("Podução anual média de cerveja")
plt.xlabel('Mês')
plt.ylabel('Produção')
plt.show()

# Distribuição da produção
plt.figure(figsize=(14,5))

sns.boxplot(x="Ano", y="Monthly beer production", data=df)
plt.xticks(rotation=45)
plt.show()

#Decomposição de séries temporais

A decomposição de uma série temporal é o processo de separar a série em diferentes componentes: tendência, sazonalidade e componente aleatório, isto permite a detecção de padrões, identificação de tendências e sazonalidades, e auxiliando na previsão de valores da série.

<img src='https://raw.githubusercontent.com/guilhermepelegrina/Mackenzie/main/Aulas/Figuras/fig_series_temporais_2.png' width="600">

*   **Tendência**  direção geral dos dados ao longo do tempo, indica se a série está aumentando, diminuindo ou permanecendo estável ao longo de um período de tempo.

*   **Sazonalidade** refere-se a padrões que se repetem em períodos fixos de tempo, como estações do ano, dias da semana ou meses.

*   **Componente aleatório (erro)**: representa a variabilidade não explicada pela tendência e sazonalidade.



Observação:

*  Quando a variação sazonal parece constante (ela não muda quando o valor da série temporal aumenta): Devemos usar o modelo aditivo (`model='additive'`), dado por:

$$Valor = Tendência +  Sazonalidade + Erro$$

* Quando a variação sazonal aumenta em magnitude: Devemos usar o modelo multiplicativo (`model='multiplicative'`), dado por:

$$ Valor = Tendência \times Sazonalidade \times Erro$$

Nos códigos abaixo utilizamos uma biblioteca em Python para decompor a série temporal.

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

decomposition=seasonal_decompose(df["Monthly beer production"],
                                 model='additive',
                                 period=12,
                                 extrapolate_trend='freq')
decomposition.plot();

Para melhor visualizar as decomposição para cada período, vamos criar novas colunas nas bases de dados para incluir os valores obtidos.

In [ ]:
# Obtendo os componentes da decomposição

trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

# Incluindo no DataFrame

df["trend"]=trend
df["seasonal"]=seasonal
df["residual"]=residual

df

Reparem que cada valor da produção é dada pela soma dos componentes de tendência, sazonalidade e resíduo. Isso acontece pelo modelo aditivo. No caso do multiplicativo, o valor da produção seria dado pela multiplicação dos valores da decomposição.